# Ансамблевые модели

## Задача классификации 

В этом практическом задании вы научитесь работать с ансамблевыми моделями. Мы начнем с задачи классификации итальянского вина на предмет его пренадлежности к одному из трех видов. Загрузите датасет `Wine Data Database` с помощью функции `load_wine` из модуля `sklearn.datasets`.

In [1]:
from sklearn.datasets import load_wine

X, y = load_wine(return_X_y=True)

Модель случайного леса для классификации представлена классом `RandomForestClassifier` из модуля `sklearn.ensemble`. Конструктор этого класса содержит аргумент `n_estimators`, который соответствует колличеству базовых алгоритмов в случайном лесе. Целью этого задания будет настройка этого параметра. Сравните модели случайных лесов с различным числом базовых алгоритмов `{1, 5, 10, 20}`. Что происходит с качеством случайного леса на тестовых данных при увеличении этого числа? Ответом на это задание `answer1` является лучшая оценка качества модели, округленная до трех знаков после запятой. Используйте `accuracy` как метрику качества и скользящий контроль `cross_val_score` как метод оценки качества модели. Установите параметр `cv = StratifiedKFold(4)`. Возьмите среднее значение оценки качества. Для каждой из моделей случайного леса используете `random_state=42` при создании нового экземпляра.

### *РЕШЕНИЕ*

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold


In [7]:
def get_score(model,cv):
    model = model.fit(X,y)
    cv_temp = cross_val_score(model,X,y,scoring='accuracy',cv=cv)

    return cv_temp.mean()
    

In [15]:
def best_score(dict_scores):
    # Превращаем словарь  в список кортежей
    temp_scores = list(dict_scores.items())
    # Сортируем по значению
    temp_scores.sort(key=lambda x: x[1])
    best_result = temp_scores[-1][1]
    return round(best_result,3)
    

In [16]:
# Создаем список содержащий проверяемые значения количества алгоритмов
list_estimators = [1, 5, 10, 20]
dict_scores = {}
# Параметры разбиения при кросс-валидации
cv = StratifiedKFold(n_splits=4)
# В цикле создаем модели с различными параметрами
for value_estimators in list_estimators:
    random_forest = RandomForestClassifier(n_estimators=value_estimators,
                                          random_state=42)
    score = get_score(random_forest,cv)
    dict_scores[value_estimators] = score 
    
answer1 = best_score(dict_scores)
    

Далее сравните модель градиентного бустинга `GradientBoostingClassifier` из `sklearn.ensemble` с логистической регрессией `LogisticRegression` из `sklearn.linear_model` на этой выборке. Используете параметр `random_state=42` при создании экземпляров классов. Какая из моделей работает лучше? Приведите лучшую оценку, округленную до трех знаков после запятой, в качестве ответа `answer2` на это задание. Какие выводы из этого можно сделать?

### *РЕШЕНИЕ*

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression


In [20]:
# Создаем список где будут хранится 2 значения
list_grad_log = []
# Градиентный бустинг
gradient_boosting = GradientBoostingClassifier(random_state=42)
list_grad_log.append(get_score(gradient_boosting,cv))
list_grad_log

[0.9219638242894057]

In [21]:
# Логистическая регрессия
logistic_regresssion = LogisticRegression(random_state=42)
list_grad_log.append(get_score(logistic_regresssion,cv))
list_grad_log

C:\ProgramData\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\envs\coursera

[0.9219638242894057, 0.9611111111111111]

In [22]:
# Выводим ответ
answer2 = round(max(list_grad_log),3)
answer2

0.961

## Задача регрессии

Загрузите уже известную вам выборку `Boston House Prices` и разделите ее случайным образом на тренировочную и тестовую выборку. Для этого используете функцию `train_test_split` с параметрами `random_state=54` и `test_size=0.33`. Мы будем сравнивать 4 модели: `RandomForestRegressor`, `GradientBoostingRegressor` из `sklearn.ensemble`, а так же Гребневую регрессию и ЛАССО (`Ridge`, `Lasso` из `sklearn.linear_model`). Обучите каждую модель на тренировочной выборке с параметром `random_state=42` в конструкторе. Какая из моделей показывает наименьшее значение среднеквадратической ошибки на тестовых данных? В качестве ответа `answer3` приведите это значение, округленное до двух цифр после запятой.

### *РЕШЕНИЕ*

In [44]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [45]:
boston = load_boston()

In [52]:
X_train, X_test, y_train, y_test = train_test_split(boston.data,boston.target,
                                                   random_state=54,
                                                   test_size=0.33)

In [54]:
# Функция для получения минимального значения среднеквадратической ошибки
# Получает на вход имя класса и словарь параметров

def get_min(model_cls,**kwargs):
    model = model_cls(**kwargs)
    # Обучаем модель
    model.fit(X_train,y_train)
    
    # Предсказываем
    predictions = model.predict(X_test)
    mean_squared = mean_squared_error(y_test,predictions)
    return mean_squared
    

In [62]:
list_alg = [RandomForestRegressor,GradientBoostingRegressor,
           Ridge, Lasso]
dict_alg = {}
# Перебираем классы и создаем в цикле модели
RANDOM_STATE = 42
for alg in list_alg:
    score = get_min(alg,random_state=RANDOM_STATE)
    dict_alg[alg] = score

# Находим наименьшее значение  
sorted_list_alg = list(dict_alg.items())
sorted_list_alg.sort(key=lambda x:x[1])
answer3 = round(sorted_list_alg[0][1],2)
answer3

C:\ProgramData\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


8.51

In [58]:
dict_alg

{sklearn.ensemble.forest.RandomForestRegressor: 10.279030538922157,
 sklearn.ensemble.gradient_boosting.GradientBoostingRegressor: 8.509604200855387,
 sklearn.linear_model.ridge.Ridge: 23.795716055531468,
 sklearn.linear_model.coordinate_descent.Lasso: 26.917061689676302}

# Строка с ответами

In [63]:
output = """Best score (random forest) {0:.3f}
Best score (other algorithms) {1:.3f}
Best score (regression) {2:.2f}"""
print(output.format(answer1, answer2, answer3))

Best score (random forest) 0.978
Best score (other algorithms) 0.961
Best score (regression) 8.51
